In [285]:
import pandas as pd
import numpy as np

from sklearn import feature_extraction
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [2]:
# seattle_cal = pd.read_csv('Data/AirBnB/seattle/calendar.csv')
seattle_list = pd.read_csv('Data/AirBnB/seattle/listings.csv')
# seattle_rev = pd.read_csv('Data/AirBnB/seattle/reviews.csv')

In [3]:
# Context manager to temporarily set options in the with statement context
def display_all(df):
    with pd.option_context('display.max_rows', 1000, 'display.max_columns', 1000):
        display(df)

In [4]:
display_all(seattle_list.head())

,id,listing_url,scrape_id,last_scraped,name,summary,space,description,experiences_offered,neighborhood_overview,notes,transit,thumbnail_url,medium_url,picture_url,xl_picture_url,host_id,host_url,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_thumbnail_url,host_picture_url,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,market,smart_location,country_code,country,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,has_availability,availability_30,availability_60,availability_90,availability_365,calendar_last_scraped,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,requires_license,license,jurisdiction_names,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
0,241032,https://www.airbnb.com/rooms/241032,20160104002432,2016-01-04,Stylish Queen Anne Apartment,NaN,Make your self at home in this charming one-be...,Make your self at home in this charming one-be...,none,NaN,NaN,NaN,NaN,NaN,https://a1.muscache.com/ac/pictures/67560560/c...,NaN,956883,https://www.airbnb.com/users/show/956883,Maija,2011-08-11,"Seattle, Washington, United States","I am an artist, interior designer, and run a s...",within a few hours,96%,100%,f,https://a0.muscache.com/ac/users/956883/profil...,https://a0.muscache.com/ac/users/956883/profil...,Queen Anne,3.0,3.0,"['email', 'phone', 'reviews', 'kba']",t,t,"Gilman Dr W, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,98119,Seattle,"Seattle, WA",US,United States,47.636289,-122.371025,t,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,""Cable TV"",Internet,""Wireless Internet"",""A...",NaN,$85.00,NaN,NaN,NaN,NaN,2,$5.00,1,365,4 weeks ago,t,14,41,71,346,2016-01-04,207,2011-11-01,2016-01-02,95.0,10.0,10.0,10.0,10.0,9.0,10.0,f,NaN,WASHINGTON,f,moderate,f,f,2,4.07
1,953595,https://www.airbnb.com/rooms/953595,20160104002432,2016-01-04,Bright & Airy Queen Anne Apartment,Chemically sensitive? We've removed the irrita...,"Beautiful, hypoallergenic apartment in an extr...",Chemically sensitive? We've removed the irrita...,none,"Queen Anne is a wonderful, truly functional vi...",What's up with the free pillows? Our home was...,"Convenient bus stops are just down the block, ...",https://a0.muscache.com/ac/pictures/14409893/f...,https://a0.muscache.com/im/pictures/14409893/f...,https://a0.muscache.com/ac/pictures/14409893/f...,https://a0.muscache.com/ac/pictures/14409893/f...,5177328,https://www.airbnb.com/users/show/5177328,Andrea,2013-02-21,"Seattle, Washington, United States",Living east coast/left coast/overseas. Time i...,within an hour,98%,100%,t,https://a0.muscache.com/ac/users/5177328/profi...,https://a0.muscache.com/ac/users/5177328/profi...,Queen Anne,6.0,6.0,"['email', 'phone', 'facebook', 'linkedin', 're...",t,t,"7th Avenue West, Seattle, WA 98119, United States",Queen Anne,West Queen Anne,Queen Anne,Seattle,WA,98119,Seattle,"Seattle, WA",US,United States,47.639123,-122.365666,t,Apartment,Entire home/apt,4,1.0,1.0,1.0,Real Bed,"{TV,Internet,""Wireless Internet"",Kitchen,""Free...",NaN,$150.00,"$1,000.00","$3,000.00",$100.00,$40.00,1,$0.00,2,90,today,t,13,13,16,291,2016-01-04,43,2013-08-19,2015-12-29,96.0,10.0,10.0,10.0,10.0,10.0,10.0,f,NaN,WASHINGTON,f,strict,t,t,6,1.48
2,3308979,https://www.airbnb.com/rooms/3308979,20160104002432,2016-01-04,New Modern House-Amazi

In [5]:
seattle_list.shape

(3818, 92)

In [6]:
seattle_list.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', '

Looking at the listing file, we see 92 variables with 3,818 records

In [291]:
df = seattle_list.copy()

## Drop variables with same value in each and every row

In [292]:
# This loop takes in all columns and returns true if the value of a column is the same all over the dataframe
empty_cols = []
for col in df.columns:
    if np.all(df.groupby(col).id.count() == 3818):
        empty_cols.append(col)

In [293]:
# These are the columns which are the same all over the records and add no value and has to be dropped
empty_cols

['scrape_id',
 'last_scraped',
 'experiences_offered',
 'market',
 'country_code',
 'country',
 'has_availability',
 'calendar_last_scraped',
 'requires_license',
 'license',
 'jurisdiction_names']

In [294]:
# Drop these 11 columns from dataframe
df.drop(empty_cols, axis=1, inplace=True)

In [295]:
df.shape

(3818, 81)

## Drop variables URL

It seems that every lisitng has its unique URL (totally expected) and can be dropped from the dataframe

In [296]:
col_url = df.columns[df.columns.str.contains('_url')]

In [297]:
col_url

Index(['listing_url', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_url', 'host_thumbnail_url', 'host_picture_url'],
      dtype='object')

In [298]:
df.drop(col_url, axis=1, inplace=True)

In [299]:
df.shape

(3818, 73)

## Name Variable

I was thinking that the names shall be unique because no one wants to have his or her listing with the same of a competitor especially when they are in the same state and maybe city because a bad review or word of mouth can be harmful

> We hve 51 listings with the same name

In [300]:
# We have 51 listings with the same name
df.name.value_counts()[:25].sum()

51

> Is it an issue in the extration or do 2 different listing share the same name?

In [301]:
df.name.value_counts()[:25]

Capitol Hill Apartment                 3
Capitol Hill Master Bedroom            2
Center UW Blast Internet New #102      2
Sans souci in Seattle                  2
Bedroom in artistic Northwest home     2
2 Baths W/D,View walk UW (302)         2
Private room in West Seattle           2
Privacy & gorgeous panoramic views!    2
Ravenna Retreat                        2
Private bath walk UW U Village(202)    2
Luxury Apt/Downtown West Seattle       2
A Quiet Room in a House by the Lake    2
Cozy Ballard Apartment                 2
Cozy studio in Capitol Hill!           2
In-City Suite Retreat                  2
comfy room&convenient location（中英）     2
Downtown studio close to everything    2
Fabulous Seattle Modern                2
Downtown Seattle Pike Place Market     2
"THE 5-STAR HOUSE"                     2
Secluded Setting in North Seattle      2
Vintage Apt in Heart of CapitolHill    2
Seattle Downtown Convention Center     2
Capitol Hill Studio Apartment          2
Charming Craftsm

In [302]:
df.groupby(['name', 'id']).id.count().sort_values(ascending=False).head()

name                             id     
• West Seattle Crash Pad •       8102349    1
Cozy Room West Seattle           6888107    1
Cozy Seattle Bungalow            6729818    1
Cozy Seattle Bedroom             9823290    1
Cozy Seattle Apartment Near UW   202260     1
Name: id, dtype: int64

In [303]:
display_all(df[df.name == 'comfy room&convenient location（中英）'])

,id,name,summary,space,description,neighborhood_overview,notes,transit,host_id,host_name,host_since,host_location,host_about,host_response_time,host_response_rate,host_acceptance_rate,host_is_superhost,host_neighbourhood,host_listings_count,host_total_listings_count,host_verifications,host_has_profile_pic,host_identity_verified,street,neighbourhood,neighbourhood_cleansed,neighbourhood_group_cleansed,city,state,zipcode,smart_location,latitude,longitude,is_location_exact,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,amenities,square_feet,price,weekly_price,monthly_price,security_deposit,cleaning_fee,guests_included,extra_people,minimum_nights,maximum_nights,calendar_updated,availability_30,availability_60,availability_90,availability_365,number_of_reviews,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,instant_bookable,cancellation_policy,require_guest_profile_picture,require_guest_phone_verification,calculated_host_listings_count,reviews_per_month
1958,9814376,comfy room&convenient location（中英）,"clean and comfy bedroom&bathroom,located in to...","Big,quiet and clean room in lovely townhouse,r...","clean and comfy bedroom&bathroom,located in to...",Located in townhouse and features a wide array...,"For some reasons,the owner is not sleep in the...","clean and comfy bedroom&bathroom,located in to...",50557855,Hua,2015-12-05,US,NaN,within a day,100%,NaN,f,Holly Park,2.0,2.0,"['email', 'phone', 'google', 'jumio']",t,t,"South Holly Park Drive, Seattle, WA 98118, Uni...",Holly Park,South Beacon Hill,Beacon Hill,Seattle,WA,98118,"Seattle, WA",47.535245,-122.284865,t,Townhouse,Private room,2,1.0,1.0,1.0,Real Bed,"{""Wireless Internet"",""Free Parking on Premises...",NaN,$60.00,NaN,NaN,$100.00,$15.00,1,$0.00,1,1125,a week ago,30,60,90,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,moderate,f,f,2,NaN
2052,9863457,comfy room&convenient location（中英）,"clean and comfy bedroom&bathroom,located in to...","Big,quiet and clean room in lovely townhouse,r...","clean and comfy bedroom&bathroom,located in to...",Located in townhouse and features a wide array...,"The guest can use the whole building,including...","clean and comfy bedroom&bathroom,located in to...",50557855,Hua,2015-12-05,US,NaN,within a day,100%,NaN,f,Holly Park,2.0,2.0,"['email', 'phone', 'google', 'jumio']",t,t,"South Holly Park Drive, Seattle, WA 98118, Uni...",Holly Park,Holly Park,Beacon Hill,Seattle,WA,98118,"Seattle, WA",47.536681,-122.286116,t,Townhouse,Private room,2,1.0,1.0,1.0,Real Bed,"{""Wireless Internet"",""Free Parking on Premises...",NaN,$50.00,NaN,NaN,$100.00,$15.00,1,$0.00,1,1125,a week ago,30,60,90,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,f,flexible,f,f,2,NaN


Although after manual inspection of these properties, it seems that these listings are not duplicates due to an error in data extarction but it is clear that although they are very similar, these are 2 distinct listings and can't be dropped, but for analysis purposes I believe the name holds little value and can be dropped especially that it is mostly a combination of the apartment or room location and some features 

In [304]:
df.drop('name', axis=1, inplace=True)

## Null values

In [305]:
df.columns[df.isnull().sum()/len(df) > .75]

Index(['square_feet'], dtype='object')

In [306]:
df.drop(['square_feet'], axis=1,inplace=True)

In [307]:
df.shape

(3818, 71)

## Checking for redundant variables

For a proper analysis, I need to make sure that there are no redundant variables 

Since some variables will look different but essentially they are the same but abbreviated, I will factorize all the variables

In [308]:
factor_df =  pd.DataFrame()

for col in df.columns:
    factor_df[col] = df[col].factorize()[0]

In [309]:
# loop over the encoded dataframe against itself to see if there are any columns with similar values
dupl_dict = {}
for index, col_1 in enumerate(factor_df.columns):
    for col_2 in factor_df.columns[index + 1:]:
        if np.all(factor_df[col_1] == factor_df[col_2]):
            dupl_dict[col_1] = col_2

In [310]:
dupl_dict

{'id': 'longitude',
 'host_listings_count': 'host_total_listings_count',
 'city': 'smart_location',
 'latitude': 'longitude'}

We have some duplicates, some are expected since when encoding, every number is given a code and columns like latitude, longitude and id will seem similar because they have totally distinct values and when enoded will look the same, but the value of this exercise is visible in the below variables

In [311]:
df.host_listings_count.value_counts().head(3)

1.0    2179
2.0     620
3.0     261
Name: host_listings_count, dtype: int64

In [312]:
df.host_total_listings_count.value_counts().head(3)

1.0    2179
2.0     620
3.0     261
Name: host_total_listings_count, dtype: int64

In [313]:
df.city.value_counts()

Seattle                  3810
West Seattle                2
Seattle                     2
seattle                     1
Phinney Ridge Seattle       1
西雅图                         1
Ballard, Seattle            1
Name: city, dtype: int64

In [314]:
df.smart_location.value_counts()

Seattle, WA                  3810
Seattle , WA                    2
West Seattle, WA                2
Ballard, Seattle, WA            1
seattle, wa                     1
Phinney Ridge Seattle, WA       1
西雅图, WA                         1
Name: smart_location, dtype: int64

I will drop one of these pair of duplicate variables

In [315]:
df.drop(['smart_location', 'host_total_listings_count'], axis=1, inplace=True)

In [316]:
df.shape

(3818, 69)

## Drop some variables that upon eyeballing looks useless

Still looking at city it hold no value as it is all data from Seattle, same for state

In [317]:
df.state.value_counts()

WA    3817
wa       1
Name: state, dtype: int64

In [318]:
df.drop(['state', 'city'], axis=1, inplace=True)

Drop neighbourhood as it is the same as neighbourhood_cleansed with missing values

In [319]:
df.drop('neighbourhood', axis=1, inplace=True)

In [320]:
df.shape

(3818, 66)

#### Going from 92 to 66 variables was quite simple because we simply dropped all redundant, empty and useless variables and here is the dataframe in its current form

Now it gets tricky because we have free text variables like summary and space which looks useless but indeed very useful, and we have some variables like zipcode and host_verifications looks empty but can be useful upon slicing

## Treating Host Name

In [321]:
# Couples for 1 and Singles for 0
pattern = '&|And'
df.loc[df.host_name.str.contains(pattern) == True, 'host_name'] = 'Couples'
df.loc[df.host_name.str.contains(pattern) == False, 'host_name'] = 'Singles'

## ZIPCODE

In [322]:
# Fix the error in this specific ZipCode manually
df.loc[df.zipcode == '99\n98122', 'zipcode'] = '98122'

## Street

In [323]:
# Drop all the rest from street as they are visible in other variables like ZipCode
df.street = df.street.str.split(',', expand=True)[0].copy()

In [324]:
df.street.value_counts().head()

1st Avenue          55
2nd Avenue          35
Western Avenue      33
10th Avenue East    29
12th Avenue East    29
Name: street, dtype: int64

## Text Variables

The below are a bit challenging because they are free text but definitly we can extract some useful keywords

In [325]:
text_var = list(df[['summary', 'space', 'description', \
               'neighborhood_overview', 'notes', 'transit', 'host_location', 'host_about']].columns)

In [326]:
# Before any treatment, I'll impute the missing varbiales in each record
# I will impute with 'No Info' because this will help me track the value of these texts if useful

df[text_var] = df[text_var].fillna('no info')

1. Create a variable with the number of characters
2. Create size of distinct words
3. Add frequency of top 25 words

In [327]:
# 1. Create a variable with the number of characters
for col in text_var:
    df[col+'_size'] = df[col].str.len()

In [328]:
# 2. Create size of distinct words
for col in text_var:
    df[col+'_count'] = df[col].str.lower().str.split().str.len()

In [329]:
# 3. Add frequency of top 25 words after dropping all the stopwords
txt_vect = feature_extraction.text.TfidfVectorizer(max_features=25, stop_words='english')

for col in text_var:
    matrix = txt_vect.fit_transform(df[col])
    matrix_df = pd.DataFrame(matrix.toarray(), columns=txt_vect.get_feature_names())
    
    for i in matrix_df:
        df[col+'_tf_'+str(i)] = matrix_df[i]

In [330]:
# Drop all these text variables
df.drop(text_var, axis=1, inplace=True)

In [331]:
len(df.columns)

274

In [333]:
display_all(df.head().T)

,0,1,2,3,4
id,241032,953595,3308979,7421966,278830
host_id,956883,5177328,16708587,9851441,1452570
host_name,Singles,Singles,Singles,Singles,Singles
host_since,2011-08-11,2013-02-21,2014-06-12,2013-11-06,2011-11-29
host_response_time,within a few hours,within an hour,within a few hours,NaN,within an hour
host_response_rate,96%,98%,67%,NaN,100%
host_acceptance_rate,100%,100%,100%,NaN,NaN
host_is_superhost,f,t,f,f,f
host_neighbourhood,Queen Anne,Queen Anne,Queen Anne,Queen Anne,Queen Anne
host_listings_count,3,6,2,1,2


## Host_verifications

In [338]:
# Get list of ways to contact each of the 
vectorizer = CountVectorizer()
tk_count = vectorizer.fit_transform(df['host_verifications'])
contact_list = vectorizer.get_feature_names()
contact_list

['amex',
 'email',
 'facebook',
 'google',
 'jumio',
 'kba',
 'linkedin',
 'manual_offline',
 'manual_online',
 'none',
 'phone',
 'photographer',
 'reviews',
 'sent_id',
 'weibo']

In [339]:
# Here I will break down the communication channels into new columns

contact_df = pd.DataFrame(tk_count.toarray(), columns=vectorizer.get_feature_names())
    
for i in contact_df:
    df[col+'_tf_'+ str(i)] = contact_df[i]

In [343]:
df.drop('host_verifications', axis=1, inplace=True)

In [344]:
df.shape

(3818, 288)

In [334]:
df.columns[:40]

Index(['id', 'host_id', 'host_name', 'host_since', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_neighbourhood', 'host_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'zipcode',
       'latitude', 'longitude', 'is_location_exact', 'property_type',
       'room_type', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
       'bed_type', 'amenities', 'price', 'weekly_price', 'monthly_price',
       'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',
       'minimum_nights', 'maximum_nights', 'calendar_updated',
       'availability_30', 'availability_60'],
      dtype='object')

In [349]:
df.amenities.loc[0], df.amenities.loc[5], df.amenities.loc[10]

('{TV,"Cable TV",Internet,"Wireless Internet","Air Conditioning",Kitchen,Heating,"Family/Kid Friendly",Washer,Dryer}',
 '{"Wireless Internet","Free Parking on Premises",Heating,"Smoke Detector",Essentials,Shampoo}',
 '{TV,"Cable TV",Internet,"Wireless Internet",Kitchen,"Free Parking on Premises",Heating,Washer,Dryer,"Smoke Detector","Carbon Monoxide Detector","First Aid Kit","Safety Card","Fire Extinguisher",Essentials,Shampoo,"24-Hour Check-in",Hangers,"Hair Dryer",Iron,"Laptop Friendly Workspace"}')

## DATES

# WHAT IS THE RESPONSE VARIABLE


1. Can you describe the vibe of each Seattle neighborhood using listing descriptions?
2. What are the busiest times of the year to visit Seattle? By how much do prices spike?
3. Is there a general upward trend of both new Airbnb listings and total Airbnb visitors to Seattle?
4. What are the most important variables that affect the number of reviews and ratings?
5. Can we extarct the number of bookings per listing from the number of reviews divided by host_since